In [1]:
import time
import random
from typing import Any, Sequence, Mapping, Optional

In [2]:
# example of fitting a gaussian mixture model with expectation maximization
import numpy as np
from sklearn import mixture
from scipy import stats
from scipy import linalg
from sklearn import model_selection
import pandas as pd
import gymnasium as gym

In [3]:
from rlplg.environments import gridworld, iceworld, redgreen

In [4]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns

## Tiling

In [5]:
"""
Tile Coding Software version 3.0beta
by Rich Sutton
based on a program created by Steph Schaeffer and others
External documentation and recommendations on the use of this code is available in the 
reinforcement learning textbook by Sutton and Barto, and on the web.
These need to be understood before this code is.

This software is for Python 3 or more.

This is an implementation of grid-style tile codings, based originally on
the UNH CMAC code (see http://www.ece.unh.edu/robots/cmac.htm), but by now highly changed. 
Here we provide a function, "tiles", that maps floating and integer
variables to a list of tiles, and a second function "tiles-wrap" that does the same while
wrapping some floats to provided widths (the lower wrap value is always 0).

The float variables will be gridded at unit intervals, so generalization
will be by approximately 1 in each direction, and any scaling will have 
to be done externally before calling tiles.

Num-tilings should be a power of 2, e.g., 16. To make the offsetting work properly, it should
also be greater than or equal to four times the number of floats.

The first argument is either an index hash table of a given size (created by (make-iht size)), 
an integer "size" (range of the indices from 0), or nil (for testing, indicating that the tile 
coordinates are to be returned without being converted to indices).
"""

basehash = hash

class IHT:
    "Structure to handle collisions"
    def __init__(self, sizeval):
        self.size = sizeval                        
        self.overfullCount = 0
        self.dictionary = {}

    def __str__(self):
        "Prepares a string for printing whenever this object is printed"
        return "Collision table:" + \
               " size:" + str(self.size) + \
               " overfullCount:" + str(self.overfullCount) + \
               " dictionary:" + str(len(self.dictionary)) + " items"

    def count(self):
        return len(self.dictionary)
    
    def fullp(self):
        return len(self.dictionary) >= self.size
    
    def getindex(self, obj, readonly=False):
        d = self.dictionary
        if obj in d: return d[obj]
        elif readonly: return None
        size = self.size
        count = self.count()
        if count >= size:
            # TODO: Fail
            if self.overfullCount==0: print('IHT full, starting to allow collisions')
            self.overfullCount += 1
            return basehash(obj) % self.size
        else:
            d[obj] = count
            return count

def hashcoords(coordinates, m, readonly=False):
    if type(m)==IHT: return m.getindex(tuple(coordinates), readonly)
    if type(m)==int: return basehash(tuple(coordinates)) % m
    if m==None: return coordinates

from math import floor, log
from itertools import zip_longest

def tiles (ihtORsize, numtilings, floats, ints=[], readonly=False):
    """returns num-tilings tile indices corresponding to the floats and ints"""
    qfloats = [floor(f*numtilings) for f in floats]
    Tiles = []
    for tiling in range(numtilings):
        tilingX2 = tiling*2
        coords = [tiling]
        b = tiling
        for q in qfloats:
            coords.append( (q + b) // numtilings )
            b += tilingX2
        coords.extend(ints)
        Tiles.append(hashcoords(coords, ihtORsize, readonly))
    return Tiles

def tileswrap (ihtORsize, numtilings, floats, wrapwidths, ints=[], readonly=False):
    """returns num-tilings tile indices corresponding to the floats and ints, wrapping some floats"""
    qfloats = [floor(f*numtilings) for f in floats]
    Tiles = []
    for tiling in range(numtilings):
        tilingX2 = tiling*2
        coords = [tiling]
        b = tiling
        for q, width in zip_longest(qfloats, wrapwidths):
            c = (q + b%numtilings) // numtilings
            coords.append(c%width if width else c)
            b += tilingX2
        coords.extend(ints)
        Tiles.append(hashcoords(coords, ihtORsize, readonly))
    return Tiles



In [6]:
def pow2geq(lb):
    exp = 1
    while True:
        rs = np.power(2, exp)
        if rs >= lb:
            break
        exp += 1
    return rs

In [7]:
def solve_least_squares(
    matrix: np.ndarray, rhs: np.ndarray
) -> np.ndarray:
    try:
        solution, _, _, _ = linalg.lstsq(a=matrix, b=rhs, lapack_driver="gelsy")
        return solution  # type: ignore
    except linalg.LinAlgError as err:
        # the computation failed, likely due to the matix being unsuitable (no solution).
        raise ValueError("Failed to solve linear system") from err

In [8]:
def rmse(v_pred: np.ndarray, v_true: np.ndarray, axis: int):
    if np.shape(v_pred) != np.shape(v_true):
        raise ValueError(
            f"Tensors have different shapes: {np.shape(v_pred)} != {np.shape(v_true)}"
        )
    return np.sqrt(
        np.sum(np.power(v_pred - v_true, 2.0), axis=axis) / np.shape(v_pred)[axis]
    )

In [9]:
class Tiles:
    def __init__(
        self, dims_min: np.ndarray, dims_max: np.ndarray, 
        tiling_dim: int, num_tilings: Optional[int] = None
    ):
        assert isinstance(dims_min, np.ndarray)
        assert isinstance(dims_max, np.ndarray)
        self.dims_max = dims_max
        self.dims_min = dims_min
        self.tiling_dim = tiling_dim
        self.wrapwidths = [tiling_dim] * np.size(dims_min)
    
        # num tilings should a power of 2
        # and at least 4 times greater than
        # the number of dimensions
        self.num_tilings = num_tilings or pow2geq(np.size(dims_min) * 4)
        self.max_size = (tiling_dim ** np.size(dims_min)) * self.num_tilings
        print("Num tilings", self.num_tilings, "\n", "Flat dim:", self.max_size)
        self.iht = IHT(self.max_size)

    def __call__(self, xs):
        xs_scaled_01 = (xs - self.dims_min) / (self.dims_max - self.dims_min)
        repr = np.zeros(shape=self.max_size)
        idx = tileswrap(
            self.iht, 
            self.num_tilings, 
            xs_scaled_01 * self.tiling_dim,
            self.wrapwidths
        )
        repr[idx] = 1
        return repr    

## Control with SARSA

In [10]:
def collection_traj_data(env, steps: int):
    obs, _ = env.reset()
    step = 0
    buffer = []
    while step < steps:
        action = env.action_space.sample()
        next_obs, rew, term, trunc, _,  = env.step(action)
        step += 1
        buffer.append((obs, action, rew))
        obs = next_obs
        if term or trunc:
            obs, _ = env.reset()
    return buffer

In [11]:
def delay_reward_data(buffer, delay: int, sample_size: int):
    obs = np.stack([example[0] for example in buffer])
    action = np.stack([example[1] for example in buffer])
    reward = np.stack([example[2] for example in buffer])

    # repr: (m1,a1)(m2,a1)..
    mdim = obs.shape[1] * len(np.unique(action))
    num_components = obs.shape[1]

    # build samples
    mask = np.random.choice(len(obs), (sample_size, delay))
    delayed_obs = obs[mask] # batch x delay x dim
    delayed_act = action[mask]
    delayed_rew = np.sum(reward[mask], axis=1) # batch x delay -> batch    
    
    rhat_matrix = np.zeros(shape=(len(delayed_obs), mdim))
    
    for i in range(len(delayed_obs)):
        for j in range(delay):
            c = num_components*delayed_act[i][j]
            rhat_matrix[i,c:c+num_components] += delayed_obs[i][j]

    return rhat_matrix, delayed_rew

In [12]:
def proj_obs_to_rwest_vec(buffer, sample_size: int):
    obs = np.stack([example[0] for example in buffer])
    action = np.stack([example[1] for example in buffer])
    reward = np.stack([example[2] for example in buffer])
    
    # repr: (m1,a1)(m2,a1)..
    mdim = obs.shape[1] * len(np.unique(action))
    num_components = obs.shape[1]

    # build samples
    mask = np.random.choice(len(obs), sample_size)
    delayed_obs = obs[mask] # batch x dim
    delayed_act = action[mask] # batch
    delayed_rew = reward[mask] # batch
    
    rhat_matrix = np.zeros(shape=(len(delayed_obs), mdim))
    
    for i in range(len(delayed_obs)):
        c = num_components*delayed_act[i]
        rhat_matrix[i,c:c+num_components] += delayed_obs[i]
    return rhat_matrix, delayed_rew

In [13]:
def solve_rwe(env: gym.Env, num_steps: int, sample_size: int, delay: int):
    buffer = collection_traj_data(env, steps=num_steps)
    Xd, yd = delay_reward_data(buffer, delay=delay, sample_size=sample_size)
    return buffer, solve_least_squares(Xd, yd)

In [14]:
def rwe_scatterplot(v_pred, v_true):
    _, ax = plt.subplots(figsize=(6, 6))
    df = pd.DataFrame({
        "x": v_pred,
        "y": v_true,
        "size": np.abs(v_pred - v_true)
    })
    sns.scatterplot(
        df, x="x", y="y", size="size", hue="size", s=5, color=".15"
    )
    ax.set_xlabel("yhat")
    ax.set_ylabel("ytrue")
    ax.set_title("Reward Estimation")

In [15]:
def hashtrick(xs, dim: int):
    ys = np.zeros(dim, dtype=np.int32)
    idx,  = np.where(xs == 1)
    for i in idx:
        ys[i % dim] += 1
    return ys

### Moutain Car

In [16]:
class MCObsWrapper(gym.ObservationWrapper):
    def __init__(self, env, tiling_dim: int, num_tilings: int = None):
        super().__init__(env)
        self.tiles = Tiles(
                env.observation_space.low,
                env.observation_space.high,
                tiling_dim=tiling_dim,
                num_tilings=num_tilings
            )
        self.observation_space = gym.spaces.Box(
            low=np.zeros(shape=self.tiles.max_size, dtype=np.int32), 
            high=np.ones(shape=self.tiles.max_size, dtype=np.int32)
        )

    def observation(self, obs):
        return self.tiles(obs)

In [20]:
env = MCObsWrapper(
    env=gym.make("MountainCar-v0", max_episode_steps=10000), 
    tiling_dim=8
)

def feat_transform(obs, actions: Sequence[int]):
    num_actions = len(actions)
    dim = obs.shape[0] * num_actions
    state_action_m = np.zeros(shape=(num_actions, dim))

    for idx, action in enumerate(actions):
        s_col = obs.shape[0] * action
        state_action_m[idx, s_col:s_col+obs.shape[0]] += obs
    return state_action_m


def action_values(obs, actions: Sequence[int], weights):
    state_action_m = feat_transform(obs, actions)
    return np.dot(state_action_m, weights)

# TODO: unify values and gradient fn (one call for both)
def semi_gradient_sarsa(env, alpha: float, gamma: float, epsilon: float, num_episodes: int):
    actions = tuple(range(env.action_space.n))
    weights = np.zeros(env.observation_space.shape[0] * len(actions), dtype=np.float64)
    returns = []
    
    for i in range(num_episodes):
        obs, _ = env.reset()
        state_qvalues = action_values(obs, actions, weights)
        gradients = feat_transform(obs, actions)
        rewards = 0
        # choose action
        if random.random() <= epsilon:
            action = env.action_space.sample()
        else:
            action = np.random.choice(np.flatnonzero(state_qvalues == state_qvalues.max()))

        while True:
            # greedy            
            next_obs, reward, term, trunc, _,  = env.step(action)
            rewards += reward
            
            if term or trunc:
                weights = weights + alpha * (reward - state_qvalues[action]) * gradients[action]
                break

            next_state_qvalues = action_values(next_obs, actions, weights)
            next_gradients = feat_transform(next_obs, actions)
            
            if random.random() <= epsilon:
                next_action = env.action_space.sample()
            else:
                # greedy
                next_action = np.random.choice(np.flatnonzero(next_state_qvalues == next_state_qvalues.max()))

            weights = weights + alpha * (
                reward + gamma * next_state_qvalues[next_action] - state_qvalues[action]
            ) * gradients[action]
            obs = next_obs
            action = next_action
            state_qvalues = next_state_qvalues
            gradients = next_gradients
        returns.append(rewards)
        if i % 200 == 0:
            print("Episode", i, "mean returns:", np.mean(returns[-100:]))
    return weights


def play(env, weights, num_episodes: int):
    actions = tuple(range(env.action_space.n))
    for i in range(num_episodes):
        obs, _ = env.reset()
        rewards = 0
        print("Playing ep", i + 1)
        while True:
            state_qvalues = action_values(obs, actions, weights)
            action = np.random.choice(np.flatnonzero(state_qvalues == state_qvalues.max()))
            next_obs, reward, term, trunc, _,  = env.step(action)
            rewards += reward
            obs = next_obs
            if term or trunc:
                print("Return", rewards)
                break



Num tilings 8 
 Flat dim: 512


In [21]:
weights = semi_gradient_sarsa(env, alpha=0.1, gamma=0.99, epsilon=0.2, num_episodes=5000)
play(env, weights, 3)

Episode 0 mean returns: -1230.0
Episode 200 mean returns: -442.69
Episode 400 mean returns: -597.56
Episode 600 mean returns: -721.25
Episode 800 mean returns: -721.01
Episode 1000 mean returns: -1012.24
Episode 1200 mean returns: -1176.92
Episode 1400 mean returns: -692.92
Episode 1600 mean returns: -775.17
Episode 1800 mean returns: -735.27
Episode 2000 mean returns: -1066.26
Episode 2200 mean returns: -772.62
Episode 2400 mean returns: -720.78
Episode 2600 mean returns: -725.63
Episode 2800 mean returns: -729.17
Episode 3000 mean returns: -727.12
Episode 3200 mean returns: -708.63
Episode 3400 mean returns: -737.82
Episode 3600 mean returns: -588.97
Episode 3800 mean returns: -775.25
Episode 4000 mean returns: -842.01
Episode 4200 mean returns: -892.45
Episode 4400 mean returns: -595.43
Episode 4600 mean returns: -822.0
Episode 4800 mean returns: -736.18
Playing ep 1
Return -276.0
Playing ep 2
Return -282.0
Playing ep 3
Return -265.0


In [ ]:
# TODO: refactors
# 1. Introduce features processor for (S,A), initialised for an environment (can use tiles or gaussian, etc)
# 2. Use replay generator - updates will apply at the end of an episode - consistently across algorithms
# 3. Introduce Policy classs (with fn approx interface - updates can be done internally)
# 4. Assess benefits of using torch for updates (computing gradients)
# 5. zero impute
# 6. options baseline - dynamic options
